In [1]:
# Install libraries if not installed
# !pip install html5lib
# !pip install selenium
# !pip install splinter
# !pip install webdriver_manager
# !pip install pandas
# !pip install pymongo

In [2]:
# Import Dependencies
import pandas as pd
from tqdm import tqdm
import re
import time
import pymongo
import random

from splinter import Browser
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}

In [4]:
# Get an instance of Browser
browser = Browser('chrome', **executable_path, headless=False)

In [5]:
# Set up MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [6]:
db = client.goodwill
collection = db.items

## Open Goodwill site and fill search form

- Open Goodwill site and search for computer laptops. 
- Set 90 days to go back from today. 
- Min price set to 200. 
- Click on 'This website uses cookies' dialog if present.

In [ ]:
browser.visit('https://shopgoodwill.com/search/advancedsearch')
element = browser.find_by_id('ddlCategories').first
element.select('7') # Computers and Electronics
element = browser.find_by_id('ddlCategories1').first
element.select('30') # Computers 
element = browser.find_by_id('ddlCategories2').first
element.select('176') # Laptops
# Select 'Closed Auctions'
browser.find_by_name('searchClosedAuctions')[2].check()
# Select days to go back from Today
element = browser.find_by_id('ddlClosedAuctionDaysBack').first
element.select('90') # 90 days
# Set minimum price
browser.find_by_id('txtLowPrice').fill('200')
# Wait 20 seconds before checking
time.sleep(20)
# Click on accept cookies - look for text
if browser.is_text_present('This website uses cookies to ensure you get the best experience on our website'):
    browser.links.find_by_text('Got it!').click()
# Click on Search button
element = browser.find_by_css('.button.btn.btn-primary.w-100').first
element.click()